# Seminar 3 - Security Implications of Power Management Mechanisms In Modern Processors

There are several attacks that exploit different power management vulnerabilities of modern high-performance computing systems.

<u>Example</u> "catching the flu", hardware Trojan in the power management unit that significantly degrade power efficiency.

<hr>

### Problem
**Current management mechanisms throttle instruction execution** and **adjust voltage/frequency to accommodate power-hungry instructions** (PHIs). These mechanisms **may compromise a system's confidentiality guarantees**.
### Goal
1. Understand the throttling side-effects of current management mechanisms
2. build high-capacity covert channels between otherwise isolated execution contexts
3. practically and efficiently mitigate each covert channel

### Characterization
Variable execution times and frequency changes due to running PHIs. We observe five different levels of throttling in real Intel systems
### IChannels
New covert channels that exploit side-effects of current management mechanisms on the same hardware thread, accross co-located Simultaneous Multi-Threading (SMT) threads and across different physical cores
### Evaluation
On three generations of Intel processors, IChannels provide a channel capacity:
- 2 times that of PHIs cariable latency-based covert channels
- 24 times that of power management-based covert channels

## 1 - Overview of Client Processor Architectures

Recent processors usually share the same voltage regulator (VR) and clock domain. 

A central power management unit (PMU) controls the VR using an off-chip serial voltage identification (SVID) interface and the clock phase-locked loop (PLL) using an on-chip interface.

Each CPU core has a power-gate (PG) for the entire core. Each SIMD unit (e.g. AVX-256, AVX-512) has a sperate PG.

### Load voltage and voltage guardband

The relationship between load voltage, supply voltage, and current under a given system impedence  is:
$$Vcc_{load} = Vcc - Icc * R_{LL}$$

The PMU adds voltage guardbands to Bcc to a level that keeps $VCC_{load}$ within limits.

For loads with current lower than Icc_{virus}, the voltage drop ($Icc * R_{LL}$) is smaller than when running a power-virus.

![gb](images/guardbands.png)

This leads to:

- Results in a higher load voltage than necessary
- a power loss that increases quadratically with the voltage level

## 2 - Motivation and Goal

**NetSpectre** exploits the variable execution times of PHI to create a covert channel. NetSpectre has three limitations:

- Established only between two execution contexts on the same hardware thread
- uses only a singlelevel throttling side-effect (thread is throttled or not)
- does not identify the true source of throttling (hypothesizes that the throttling is due to power gating of the PHI)


**TurboCC** exploits the core frequency throttling when executing PHIs to create cross-core covert channels.

As such:

Recent works propose limited coverts channels and inaccurate observations.

### Goal:

1. Experimentally understand the throttling side-effects of current managment mechanisms in modern processors to gain several deep insights into how these mechanisms can bbe abused by attackers

2. Build high capacity covery channels (IChannels) between otherwise isolated execution contexts

3. Practically and effectively mitigate covert channels caused by current management mechanisms

### Methodology:

Experiments on Intel processors

### Votlage Emergency Avoidance MEchanism

PHI influence supply votage. Supply voltage is tracked.

**Voltage emergency avoidance mechanism prevents the core voltage from dropping below the minimum operational voltage limit when executing PHIs**.

**Results**:

- For both desktop configuration frequencies, current is below the system limt
- Vcc exeed the voltage limit when executing AVX2 code at the highest frequency
- For both mobile frequencies, the Vcc is below the system limit

### Conclusion

Contrary to the state-of-the-art work's hypothesis:

> **The core frequency reduction that directly follows the execution of PHIs at the turbo frequency is not due to thermal management**
>
> **It is due to maximum instantaneous current limit and maximum voltage limit protection mechanisms**
>
> **Power-gating AVX execution units accounts for only ~0.1% of the total throttling time observed when executing PHIs**

## 3 - Throttling characterization

### Throttling affects SMT threads

Contrary to the state-of-the-art work's hypothesis: the 4x core IPC reduction that directly follows the execution of PHIs is not due to reduced core clock frequency of 4x. It is rather because the core blocks the bront-end to back-end uop delivery during 75% of the time.

This throttling mechanism affects both threads in an SMT core.

## 4 - IChannels Covert Channels

Thread model consist of two malicious user-level attacker applications, **sender** and **Receiver**, which cannot communicate through overt channels.

We build three high-throughput covert channels between sender and receiver that exploit throttling side-effects of current management mechanism (same HW thread, across SMT threads, across core).

Each covert channel sends 2 bits from Sender to Receiver in every transaction:
- each covert channel should wait for reset-time (c. 650 microseconds) before starting a new transaction.

### IccThreadCovert - on the same hardware thread

Exploitation of the side effect of **Multi-throttling-threads** (executing an instruction with high computational intensity results in a throttling period proportional to the difference in voltage requirements of the currently and previously executing instructions). 

### IccSMTcovert - across co-located SMT threads

Exploitation of the side-effect of **Multi-Throttling-SMT** (when a thread is throttled due to executing PHIs, the co-located SMT thread is also throttled).

Co-located hardware threads are throttled together because the throttling mechanism in the core pipeline blocks the front-end to back-end interface during three-quraters of the TP for the entire core.

The exploit builds a covert channel between Sender and REceiber. The sender executes a PHI loop with a computational intensity level depending on the values of two secret bits it wants to send. The receiver can infer the two bits sent by the sender based on the measured throttling period of the 64b loop.

### IccCoresCovert - across different physical cores

Exploitation of the side-effect of multi-throttling cores. 

When two cores execute PHI's at similar times, the throttling periods are exacerbated proportionally to the computational intensity of each PHI executed in each core.

The increase in the TP is because the PMU waits for the voltage transition for core A to complete before starting the voltage transition for core B. 

## Evaluation

### Methodology: Framework

Evaluation of IChannels on Coffee Lake and Cannon Lake (IccThreadCovert and IccCoresCovert on both processors).

### Methodology: Workloads: 

Proof-of-concept codes of each of the three IChannels covert channels

### Methodology: Comparison points

IChannels are compoted to four recent works that exploit different Power Management Mechanisms of modern processors to build covert channels.

## Conclusion

### Mitigation

Mitigation of IChannels can be achieved by hardware and software techniques:
- Fast Per-Core Voltage Regulators
- Improved Core Throttling
- New Secure Mode of Operation